In [2]:
# 抽取 实体-关系词二元组
import json
data=json.loads(open("../result/es_top20_train.json",encoding="utf8").read())

In [20]:
data[2]

{'id': 'TR002941',
 'question': '圆明园的开放时间是什么时候？',
 'attribute': ['没有找到该问题对应的知识'],
 'top20': [['5029130', '时间是什么 ||| 书名 ||| 时间是什么'],
  ['5029129', '时间是什么 ||| 别名 ||| 时间是什么'],
  ['5181735', '什么时候 ||| 别名 ||| 什么时候'],
  ['12223520', '什么时候明朝初年 ||| 别名 ||| 什么时候明朝初年'],
  ['5029136', '时间是什么 ||| 开本 ||| 32'],
  ['12223521', '什么时候明朝初年 ||| 中文名 ||| 什么时候明朝初年'],
  ['465265', '圆明园遗址公园 ||| 开放时间 ||| 1988年'],
  ['16595056', '什么时候说什么话 ||| 别名 ||| 什么时候说什么话'],
  ['16595061', '什么时候说什么话 ||| 出版时间 ||| 2008.6'],
  ['5181736', '什么时候 ||| 中文名称 ||| 什么时候'],
  ['8765481', '爱情从什么时候开始 ||| 别名 ||| 爱情从什么时候开始'],
  ['17544776', '忘了什么时候说过不放手 ||| 别名 ||| 忘了什么时候说过不放手'],
  ['28793305', '什么时候我们才学会放手 ||| 别名 ||| 什么时候我们才学会放手'],
  ['8765482', '爱情从什么时候开始 ||| 中文名 ||| 爱情从什么时候开始'],
  ['17544777', '忘了什么时候说过不放手 ||| 中文名 ||| 忘了什么时候说过不放手'],
  ['28793306', '什么时候我们才学会放手 ||| 中文名 ||| 什么时候我们才学会放手'],
  ['3372028', '都什么时候了 ||| 别名 ||| 都什么时候了'],
  ['36489788', '什么时候吻她 ||| 别名 ||| 什么时候吻她'],
  ['28113773', '开放人文·真科学：它是什么，它指什么 ||| 出版时间 ||| 2008-12-1'],
  ['176

In [3]:
def generate_prompt_QuerySummary(related_text, query: str, preprompt="") -> str:
    prompt_template = """已知问题:
        {question}，请提取问题的摘要"""

    prompt = preprompt + prompt_template.replace("{question}", query).replace("{context}", related_text)
    return prompt

In [7]:
data_tuple=[]
for unit in data:
    if '没有找到该问题对应的知识' not in unit["attribute"]:
        # print(unit)
        question = unit["question"]
        answer = ";".join(["".join([t.strip() for t in k.split("|||")[:2]]) for k in unit["attribute"]])
        data_tuple.append({"question":question,"instruction":"","answer":answer,"task_type":"tuple_extract"})

In [8]:
data_tuple[0]

{'question': '我听说过唐岐山，他是中国人并且出生在河南开封。但是，我想了解有关他出生和去世的具体日期，还有他在‘文化大革命’中的罪行。',
 'instruction': '',
 'answer': '唐岐山出生年月;唐岐山去世年月;唐岐山其他信息',
 'task_type': 'tuple_extract'}

In [9]:
data_tuple[2]

{'question': '周穆王在位多少年？',
 'instruction': '',
 'answer': '周穆王在位时间',
 'task_type': 'tuple_extract'}

In [24]:
# 抽取表格信息
def generate_prompt_QA(related_text, query: str, preprompt="") -> str:
    prompt_template = """问题:'{question}'，请从已知内容中抽取答案。如果无法从中得到答案，请说 "没有找到该问题对应的知识"。已知内容:{context}
        """

    prompt = preprompt + prompt_template.replace("{question}", query).replace("{context}", related_text)
    return prompt


In [25]:
# step
i=16
generate_prompt_QA(str(";".join([k[1] for k in data[i]["top20"]])),data[i]["question"])

'问题:\'井上雄是什么样的人物类型？\'，请从已知内容中抽取答案。如果无法从中得到答案，请说 "没有找到该问题对应的知识"。已知内容:富人的世界是什么样 ||| 别名 ||| 富人的世界是什么样;我是一个什么样的人 ||| 别名 ||| 我是一个什么样的人;什么样的人做什么样的事 ||| 别名 ||| 什么样的人做什么样的事;什么样的人做什么样的事 ||| 书名 ||| 什么样的人做什么样的事;《什么样的人赚什么样的钱》 ||| 别名 ||| 《什么样的人赚什么样的钱》;《什么样的人赚什么样的钱》 ||| 中文名 ||| 《什么样的人赚什么样的钱》;富人的世界是什么样 ||| 中文名 ||| 富人的世界是什么样;我是一个什么样的人 ||| 中文名 ||| 我是一个什么样的人;什么样的人有什么样的经历 ||| 别名 ||| 什么样的人有什么样的经历;是什么样的感情 ||| 别名 ||| 是什么样的感情;什么样的人有什么样的经历 ||| 中文名 ||| 什么样的人有什么样的经历;娶什么样的女人，嫁什么样的男人 ||| 别名 ||| 娶什么样的女人，嫁什么样的男人;是什么样的感情 ||| 中文名 ||| 是什么样的感情;井上雄 ||| 人物类型 ||| 动漫人物;什么样的人能干，什么样的人不能干 ||| 别名 ||| 什么样的人能干，什么样的人不能干;你是什么样的父母 ||| 别名 ||| 你是什么样的父母;什么样的人生是值得追求的 ||| 别名 ||| 什么样的人生是值得追求的;你是什么类型的父母 ||| 别名 ||| 你是什么类型的父母;你是什么类型的父母 ||| 书名 ||| 你是什么类型的父母;金玉彬 ||| 是什么样的人 ||| 火\n        '

In [26]:
top20 = [k[1] for k in data[i]["top20"]]
set(top20)&set(data[i]["attribute"])

{'井上雄 ||| 人物类型 ||| 动漫人物'}

In [27]:
data_table_extraction=[]
for unit in data:
    top20 = [k[1] for k in unit["top20"]]
    if not set(top20)&set(unit["attribute"]):
        answer = "没有找到该问题对应的知识"
    else:
        answer = ";".join(set(top20)&set(unit["attribute"]))
    data_table_extraction.append({"question":unit["question"],"instruction":top20,"answer":answer,"task_type":"table_extract"})
        

In [28]:
data_table_extraction[0]

{'question': '我听说过唐岐山，他是中国人并且出生在河南开封。但是，我想了解有关他出生和去世的具体日期，还有他在‘文化大革命’中的罪行。',
 'instruction': ['他说我是他的 ||| 中文名 ||| 他说我是他的',
  '他说我是他的 ||| 别名 ||| 他说我是他的',
  '我的他不是他 ||| 中文名 ||| 我的他不是他',
  '他，是我的 ||| 中文名 ||| 他,是我的',
  '他是我的传说 ||| 中文名 ||| 他是我的传说',
  '我爱的人有了他 ||| 中文名 ||| 我爱的人有了他',
  '《爱我还是他》 ||| 中文名 ||| 《爱我还是他》',
  '爱我还是他 ||| 中文名 ||| 《爱我还是他》',
  '他说她是他的命 ||| 中文名 ||| 他说她是他的命',
  '爱我还是他 ||| 中文名称 ||| 《爱我还是他》',
  '你究竟爱的是他还是我 ||| 中文名 ||| 你究竟爱的是他还是我',
  '心中想的还是他 ||| 别名 ||| 心中想的还是他',
  '心中想的还是他 ||| 中文名称 ||| 心中想的还是他',
  '他们的世界我只是过客 ||| 中文名 ||| 他们的世界我只是过客',
  '我还想他 ||| 出生日期 ||| 1981年',
  '谁是我的他 ||| 中文名 ||| 谁是我的他',
  '他是我的狼 ||| 中文名 ||| 他是我的狼',
  '你是我的他 ||| 中文名 ||| 你是我的他',
  '我爱你还是他 ||| 中文名 ||| 我爱你还是他',
  '你爱我还是他 ||| 中文名 ||| 你爱我还是他'],
 'answer': '没有找到该问题对应的知识',
 'task_type': 'table_extract'}

In [20]:
sums=0
for d in data_table_extraction:
    if d[2]!="没有找到该问题对应的知识":
        sums+=1

KeyError: 2

In [85]:
sums

335

In [29]:
# 正样本构造
import random

data_table_extraction2=[]
for unit in data:
    top20 = [k[1] for k in unit["top20"]]
    if not set(top20)&set(unit["attribute"]) and '没有找到该问题对应的知识' not in unit["attribute"]:
        answer = ";".join(unit["attribute"])
        top20=(unit["attribute"]+top20)[:20]
        random.shuffle(top20)
        data_table_extraction2.append({"question":unit["question"],"instruction":top20,"answer":answer,"task_type":"table_extract"})

In [30]:
data_sft=data_tuple+data_table_extraction+data_table_extraction2

In [31]:
random.shuffle(data_sft)

In [48]:
data_sft[19]

{'question': '我对魔兽世界中的一位角色祖尔金有了一些了解，他是森林巨魔的族人，而且他有一定的等级，还有魔法技能。然而我对他的中文名和外文名还有点疑惑。并且他是以什么身份出场的也未知，你能给我提供更多关于他的信息吗？',
 'instruction': ['祖尔金 ||| 中文名 ||| 祖尔',
  '恶魔？不，他是我的王子 ||| 中文名 ||| 恶魔?不,他是我的王子',
  '祖尔金 ||| 职业 ||| 狂暴猎头者',
  '你究竟爱的是他还是我 ||| 别名 ||| 你究竟爱的是他还是我',
  '关于他的你的和我的 ||| 中文名 ||| 关于他的你的和我的',
  '他说我是他的 ||| 中文名 ||| 他说我是他的',
  '祖尔金 ||| 出场作品 ||| 魔兽争霸2：黑潮魔兽世界',
  '我的他不是他 ||| 中文名 ||| 我的他不是他',
  '你是我的他 ||| 别名 ||| 你是我的他',
  '我的第一次给了他 ||| 中文名 ||| 我的第一次给了他',
  '我的第一次给了他 ||| 别名 ||| 我的第一次给了他',
  '等军焰 ||| 其他信息 ||| 他的性格是一位有聪明而且责任感的大哥，但是一生气来可是很可怕的',
  '你是我的他 ||| 中文名 ||| 你是我的他',
  '为什么你爱的人是他不是我 ||| 中文名 ||| 为什么你爱的人是他不是我',
  "祖尔金 ||| 外文名 ||| Zul'Jin",
  '他，是我的 ||| 中文名 ||| 他,是我的',
  '我爱的人有了他 ||| 中文名 ||| 我爱的人有了他',
  '你究竟爱的是他还是我 ||| 中文名 ||| 你究竟爱的是他还是我',
  '我爱的人有了他 ||| 别名 ||| 我爱的人有了他',
  '祖尔金 ||| 别名 ||| 祖尔金'],
 'answer': "祖尔金 ||| 别名 ||| 祖尔金;祖尔金 ||| 中文名 ||| 祖尔;祖尔金 ||| 外文名 ||| Zul'Jin;祖尔金 ||| 职业 ||| 狂暴猎头者;祖尔金 ||| 出场作品 ||| 魔兽争霸2：黑潮魔兽世界",
 'task_type': 'table_extract'}

In [49]:
with open("train_sft.json","w",encoding="utf8" ) as f:
    for unit in data_sft[:int(0.8*(len(data_sft)))]:
        f.write(json.dumps(unit,ensure_ascii=False))
        f.write("\n")

with open("dev_sft.json","w",encoding="utf8" ) as f:
    for unit in data_sft[int(0.8*(len(data_sft))):]:
        f.write(json.dumps(unit,ensure_ascii=False))
        f.write("\n")